# INFO 2950 Final Project

## Introduction:

## Data Description and Cleaning:

## Preregistration Statement: 

## Data Analysis: 

## Evaluation of Significance: 

## Conclusions: 

## Limitations: 

## Acknowledgements and Bibliography: 